In [1]:
## Seismic Analysis Impact on NR&C - Control Population
## By: Fervis Lauan
## Date: 09-29-17
##----------------------------------------------------

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import requests
from census import Census
from us import states


In [2]:
##################################
# Get Control Population
##################################


yr=2013
ctr=1
while yr<=2014:
    try:
        c = Census("85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98", year=yr)
        census_data = c.acs5.get(("NAME",
                                          "B01003_001E", 
                                          "B01002_001E",
                                          "B19013_001E",
                                          "B19301_001E",
                                          "B17001_002E",
                                          "B23025_002E",
                                          "B23025_005E",
                                          "B25077_001E",
                                          "B25064_001E",
                                          "C24010_009E",
                                          "C24010_016E",
                                          "C24010_019E",
                                          "C24010_022E",
                                          "C24010_023E",
                                          "C24010_025E",
                                          "C24010_030E",
                                          "C24010_032E",
                                          "C24010_045E",
                                          "C24010_052E",
                                          "C24010_055E",
                                          "C24010_058E",
                                          "C24010_059E",
                                          "C24010_061E",
                                          "C24010_066E",
                                          "C24010_068E",                         
                                 ), {'for': 'zip code tabulation area:*'})
        # Convert to DataFrame
        census_pd = pd.DataFrame(census_data)
        census_pd["Year"]=yr
        # Column Reordering
        census_pd = census_pd.rename(columns={"B01003_001E": "Population", 
                                              "B01002_001E": "Median Age",
                                              "B19013_001E": "Household Income",
                                              "B19301_001E": "Per Capita Income",
                                              "B17001_002E": "Poverty Count",
                                              "B23025_002E": "Employment Count",
                                              "B23025_005E": "Unemployment Count",
                                              "B25077_001E": "Median Home Value",
                                              "B25064_001E": "Median Gross Rent",                                      
                                              "C24010_009E": "Emp Male Arch Engnr",
                                              "C24010_016E": "Emp Male Health Prac",
                                              "C24010_019E": "Emp Male Service Occ",
                                              "C24010_022E": "Emp Male Fire Prev",
                                              "C24010_023E": "Emp Male Law Enfrcmnt",
                                              "C24010_025E": "Emp Male BldgGrnd Cleaning",
                                              "C24010_030E": "Emp Male Ntrl Rsrces Const",
                                              "C24010_032E": "Emp Male Const Extrctn",                                      
                                              "C24010_045E": "Emp FMale Arch Engnr",
                                              "C24010_052E": "Emp FMale Health Prac",
                                              "C24010_055E": "Emp FMale Service Occ",
                                              "C24010_058E": "Emp FMale Fire Prev",
                                              "C24010_059E": "Emp FMale Law Enfrcmnt",
                                              "C24010_061E": "Emp FMale BldgGrnd Cleaning",
                                              "C24010_066E": "Emp FMale Ntrl Rsrces Const",
                                              "C24010_068E": "Emp FMale Const Extrctn",                                                                          
                                              "NAME": "Name",                                       
                                              "zip code tabulation area": "Zip",
                                              "Year":"Year"})

        census_pd["Emp Arch Engnr"]=census_pd["Emp Male Arch Engnr"]+census_pd["Emp FMale Arch Engnr"]
        census_pd["Emp Health Prac"]=census_pd["Emp Male Health Prac"]+census_pd["Emp FMale Health Prac"]
        census_pd["Emp Service Occ"]=census_pd["Emp Male Service Occ"]+census_pd["Emp FMale Service Occ"]
        census_pd["Emp Fire Prev"]=census_pd["Emp Male Fire Prev"]+census_pd["Emp FMale Fire Prev"]
        census_pd["Emp Law Enfrcmnt"]=census_pd["Emp Male Law Enfrcmnt"]+census_pd["Emp FMale Law Enfrcmnt"]
        census_pd["Emp BldgGrnd Cleaning"]=census_pd["Emp Male BldgGrnd Cleaning"]+census_pd["Emp FMale BldgGrnd Cleaning"]
        census_pd["Emp Ntrl Rsrces Const"]=census_pd["Emp Male Ntrl Rsrces Const"]+census_pd["Emp FMale Ntrl Rsrces Const"]
        census_pd["Emp Const Extrctn"]=census_pd["Emp Male Const Extrctn"]+census_pd["Emp FMale Const Extrctn"]

        # Add in Poverty Rate (Poverty Count / Population)
        census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

        # Add in Employment Rate (Employment Count / Population)
        census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"].astype(int) / census_pd["Population"].astype(int)

        # Final DataFrame
        census_pd = census_pd[["Year",                       
                               "Name", 
                               "Zip",
                               "Population", 
                               "Median Age", 
                               "Household Income",
                               "Per Capita Income", 
                               "Poverty Count", 
                               "Poverty Rate",
                               "Employment Count", 
                               "Unemployment Rate",
                               "Median Home Value",
                               "Median Gross Rent",
                               "Emp Arch Engnr",
                               "Emp Health Prac",
                               "Emp Service Occ",
                               "Emp Fire Prev",
                               "Emp Law Enfrcmnt",
                               "Emp BldgGrnd Cleaning",
                               "Emp Ntrl Rsrces Const",
                               "Emp Const Extrctn"
                               ]]                 

        if ctr==1:
            print("Creating from year "+str(yr)+"...")
            census_pd_ctrl=census_pd
        else:
            print("appending year "+str(yr)+"...")
            census_pd_ctrl=census_pd_ctrl.append(census_pd)
        ctr+=1
    except:
        print("No Data")
    yr+=1
    

Creating from year 2013...
appending year 2014...


In [3]:
census_pd_ctrl2013=census_pd_ctrl.loc[census_pd_ctrl["Year"]==2013]
census_pd_ctrl2014=census_pd_ctrl.loc[census_pd_ctrl["Year"]==2014]

In [4]:
census_pd_ctrlmrg=pd.merge(census_pd_ctrl2013, census_pd_ctrl2014, on="Zip")
census_pd_ctrlmrg

,Year_x,Name_x,Zip,Population_x,Median Age_x,Household Income_x,Per Capita Income_x,Poverty Count_x,Poverty Rate_x,Employment Count_x,...,Median Home Value_y,Median Gross Rent_y,Emp Arch Engnr_y,Emp Health Prac_y,Emp Service Occ_y,Emp Fire Prev_y,Emp Law Enfrcmnt_y,Emp BldgGrnd Cleaning_y,Emp Ntrl Rsrces Const_y,Emp Const Extrctn_y
0,2013,ZCTA5 00601,00601,18450.0,36.6,12041.0,7380.0,10816.0,58.623306,5698.0,...,105400.0,367.0,23.0,226.0,751.0,126.0,128.0,141.0,595.0,389.0
1,2013,ZCTA5 00602,00602,41302.0,38.6,15663.0,8463.0,22409.0,54.256452,15738.0,...,91200.0,411.0,122.0,553.0,2597.0,211.0,222.0,948.0,1393.0,680.0
2,2013,ZCTA5 00603,00603,53683.0,38.9,15485.0,9176.0,26220.0,48.842278,16903.0,...,128700.0,369.0,212.0,673.0,2623.0,375.0,358.0,814.0,1457.0,587.0
3,2013,ZCTA5 00606,00606,6591.0,37.3,15019.0,6383.0,3721.0,56.455773,2021.0,...,105800.0,326.0,10.0,35.0,606.0,19.0,95.0,222.0,292.0,182.0
4,2013,ZCTA5 00610,00610,28963.0,39.2,16707.0,7892.0,14569.0,50.302110,9323.0,...,113700.0,419.0,128.0,260.0,1790.0,215.0,91.0,535.0,824.0,306.0
5,2013,ZCTA5 00612,00612,68055.0,38.5,17848.0,10188.0,30695.0,45.103225,21968.0,...,103400.0,411.0,194.0,1182.0,3293.0,595.0,458.0,864.0,1348.0,573.0
6,2013,ZCTA5 00616,00616,10183.0,40.9,15342.0,8767.0,4808.0,47.215948,3154.0,...,98800.0,454.0,0.0,186.0,780.0,256.0,112.0,112.0,267.0,108.0
7,2013,ZCTA5 00617,00617,24879.0,36.2,14982.0,7939.0,13362.0,53.707946,6839.0,...,98400.0,446.0,69.0,259.0,1085.0,169.0,123.0,305.0,450.0,131.0
8,2013,ZCTA5 00622,00622,6165.0,42.0,14281.0,8751.0,3700.0,60.016221,1522.0,...,139800.0,564.0,0.0,76.0,408.0,112.0,87.0,59.0,91.0,35.0
9,2013,ZCTA5 00623,00623,44704.0,39.7,17389.0,9326.0,21322.0,47.695956,12805.0,...,113800.0,397.0,122.0,558.0,2078.0,283.0,253.0,489.0,788.0,276.0


In [5]:
census_pd_ctrlmrg=census_pd_ctrlmrg.loc[(census_pd_ctrlmrg["Population_x"]>=11000) & (census_pd_ctrlmrg["Population_x"]<=50000) & (census_pd_ctrlmrg["Emp Const Extrctn_x"]>0) & (census_pd_ctrlmrg["Emp Const Extrctn_y"]>0)]
census_pd_ctrlmrg

,Year_x,Name_x,Zip,Population_x,Median Age_x,Household Income_x,Per Capita Income_x,Poverty Count_x,Poverty Rate_x,Employment Count_x,...,Median Home Value_y,Median Gross Rent_y,Emp Arch Engnr_y,Emp Health Prac_y,Emp Service Occ_y,Emp Fire Prev_y,Emp Law Enfrcmnt_y,Emp BldgGrnd Cleaning_y,Emp Ntrl Rsrces Const_y,Emp Const Extrctn_y
0,2013,ZCTA5 00601,00601,18450.0,36.6,12041.0,7380.0,10816.0,58.623306,5698.0,...,105400.0,367.0,23.0,226.0,751.0,126.0,128.0,141.0,595.0,389.0
1,2013,ZCTA5 00602,00602,41302.0,38.6,15663.0,8463.0,22409.0,54.256452,15738.0,...,91200.0,411.0,122.0,553.0,2597.0,211.0,222.0,948.0,1393.0,680.0
4,2013,ZCTA5 00610,00610,28963.0,39.2,16707.0,7892.0,14569.0,50.302110,9323.0,...,113700.0,419.0,128.0,260.0,1790.0,215.0,91.0,535.0,824.0,306.0
7,2013,ZCTA5 00617,00617,24879.0,36.2,14982.0,7939.0,13362.0,53.707946,6839.0,...,98400.0,446.0,69.0,259.0,1085.0,169.0,123.0,305.0,450.0,131.0
9,2013,ZCTA5 00623,00623,44704.0,39.7,17389.0,9326.0,21322.0,47.695956,12805.0,...,113800.0,397.0,122.0,558.0,2078.0,283.0,253.0,489.0,788.0,276.0
10,2013,ZCTA5 00624,00624,25340.0,34.2,14768.0,6761.0,15095.0,59.569850,8658.0,...,84600.0,385.0,53.0,443.0,1456.0,241.0,299.0,363.0,868.0,547.0
11,2013,ZCTA5 00627,00627,34806.0,37.9,19527.0,9112.0,15541.0,44.650348,12161.0,...,98900.0,433.0,159.0,624.0,1641.0,232.0,249.0,282.0,1135.0,587.0
13,2013,ZCTA5 00637,00637,24996.0,38.8,16430.0,8045.0,12384.0,49.543927,8237.0,...,98100.0,414.0,71.0,360.0,1249.0,200.0,313.0,246.0,545.0,309.0
14,2013,ZCTA5 00638,00638,18742.0,37.2,13860.0,6526.0,11271.0,60.137659,4735.0,...,109000.0,356.0,24.0,212.0,897.0,106.0,186.0,232.0,395.0,232.0
15,2013,ZCTA5 00641,00641,30401.0,38.7,14924.0,7479.0,16917.0,55.646196,9218.0,...,102300.0,400.0,67.0,286.0,1500.0,219.0,318.0,428.0,685.0,353.0


In [6]:
census_pd_ctrlsample=census_pd_ctrlmrg.sample(1000)


In [7]:
census_pd_ctrlsample

,Year_x,Name_x,Zip,Population_x,Median Age_x,Household Income_x,Per Capita Income_x,Poverty Count_x,Poverty Rate_x,Employment Count_x,...,Median Home Value_y,Median Gross Rent_y,Emp Arch Engnr_y,Emp Health Prac_y,Emp Service Occ_y,Emp Fire Prev_y,Emp Law Enfrcmnt_y,Emp BldgGrnd Cleaning_y,Emp Ntrl Rsrces Const_y,Emp Const Extrctn_y
27400,2013,ZCTA5 78744,78744,44375.0,27.8,41721.0,16335.0,11938.0,26.902535,22949.0,...,107200.0,982.0,278.0,275.0,6469.0,146.0,93.0,2012.0,3630.0,2828.0
6476,2013,ZCTA5 21012,21012,20781.0,40.7,101367.0,43166.0,795.0,3.825610,11891.0,...,387300.0,1693.0,214.0,715.0,1709.0,88.0,144.0,199.0,658.0,340.0
15819,2013,ZCTA5 48141,48141,25215.0,32.9,26512.0,14259.0,9547.0,37.862384,9981.0,...,51700.0,739.0,99.0,373.0,2473.0,161.0,0.0,350.0,498.0,294.0
7123,2013,ZCTA5 23224,23224,35885.0,30.8,33141.0,15740.0,11411.0,31.798802,17902.0,...,116600.0,838.0,118.0,738.0,4315.0,387.0,164.0,993.0,2073.0,1468.0
27793,2013,ZCTA5 80002,80002,18742.0,38.5,50425.0,30552.0,2861.0,15.265180,10898.0,...,212200.0,896.0,285.0,479.0,1849.0,181.0,54.0,438.0,1054.0,723.0
12194,2013,ZCTA5 37760,37760,13572.0,38.0,40647.0,19910.0,2205.0,16.246684,6650.0,...,137100.0,619.0,45.0,398.0,958.0,14.0,20.0,366.0,616.0,306.0
11515,2013,ZCTA5 35761,35761,11025.0,36.9,54856.0,26438.0,696.0,6.312925,5726.0,...,132100.0,760.0,219.0,409.0,721.0,88.0,53.0,241.0,649.0,348.0
16164,2013,ZCTA5 48854,48854,18616.0,40.7,63552.0,29580.0,1291.0,6.934895,9674.0,...,152100.0,865.0,203.0,549.0,1289.0,81.0,153.0,326.0,669.0,420.0
20869,2013,ZCTA5 61080,61080,11032.0,36.4,55172.0,24166.0,1581.0,14.331037,5701.0,...,124600.0,649.0,116.0,254.0,774.0,94.0,0.0,181.0,474.0,189.0
16418,2013,ZCTA5 49418,49418,27524.0,36.1,58814.0,27236.0,2197.0,7.982125,15030.0,...,156000.0,720.0,435.0,683.0,1898.0,155.0,33.0,399.0,804.0,396.0


In [9]:
for index,row in census_pd_ctrlsample.iterrows():
    pcchg=((row["Emp Const Extrctn_y"]+row["Emp Ntrl Rsrces Const_y"])-(row["Emp Const Extrctn_x"]+row["Emp Ntrl Rsrces Const_x"]))/(row["Emp Const Extrctn_x"]+row["Emp Ntrl Rsrces Const_x"])
    census_pd_ctrlsample.loc[index,"Emp Const Extr Change"]=pcchg
    
    


In [10]:
census_pd_ctrlsample.to_csv("./Resources/seismic_ctrl_emp_pop.csv")

    
    

In [11]:
census_pd_ctrl_ttest=census_pd_ctrlsample[["Zip","Emp Const Extr Change"]]
census_pd_ctrl_ttest

,Zip,Emp Const Extr Change
27400,78744,0.005919
6476,21012,-0.020608
15819,48141,0.358491
7123,23224,0.056384
27793,80002,-0.180728
12194,37760,-0.065856
11515,35761,-0.016765
16164,48854,-0.006387
20869,61080,-0.013393
16418,49418,-0.130435


In [12]:
census_pd_ctrl_ttest.to_csv("./Resources/seismic_ctrl_emp_pop_ttest.csv")